> **You:**
> 
> Pytorchを使ったGCNの練習問題を考えてください
> 
> **ChatGPT:**
> 
> Pytorchを使ったGCN（Graph Convolutional Network）の練習問題として、以下のような課題を考えました。
> 
> ## 練習問題: ソーシャルネットワークにおけるユーザーの分類
> ### 課題概要
> あなたは、あるソーシャルネットワークのデータを扱っています。各ユーザーはノードとして表現され、友人関係はエッジとして表現されます。各ユーザーにはいくつかの特徴が与えられており、この特徴を基にユーザーを「アクティブ」ま> たは「非アクティブ」の2つのクラスに分類するモデルを構築してください。
> 
> ### データの設定
> - ノード特徴行列: 各ノード（ユーザー）に関連する特徴を含む行列。行はノード、列は特徴量を表す。
> - 隣接行列: グラフの隣接行列。エッジ（友人関係）が存在する場合、その対応する要素は1、それ以外は0。
> - ラベル: 各ノードに関連するクラスラベル。「アクティブ」（1）と「非アクティブ」（0）。
> 
> ### 問題設定
> 1. データ準備: データセットには、1000人のユーザーの特徴（10次元のベクトル）と友人関係の情報が含まれていると仮定します。このデータをランダムに分割して、80%をトレーニングセット、20%をテストセットとします。
> 2. GCNモデルの実装:
>     - 2層のGCNを実装してください。
>     - 活性化関数としてReLUを使用します。
>     - ノードの特徴ベクトルを入力とし、出力層では各ノードが「アクティブ」である確率を出力します。
> 
> 3. 損失関数と最適化:
>     - クロスエントロピー損失関数を使用してモデルを訓練します。
>     - 最適化手法としてAdamを使用し、学習率を0.01に設定します。
> 4. モデルのトレーニング:
>     - 50エポックでモデルをトレーニングし、各エポックでトレーニングセットとテストセットの精度を評価します。
> 5. 結果の分析:
>     - トレーニングとテストの精度をプロットし、過学習の兆候がないか確認します。
>     - 最終的なテストセットの精度を報告し、モデルの性能を評価します。
> 
> ### ヒント
> - PyTorch Geometricライブラリを使用すると、GCNの実装が容易になります。このライブラリは、グラフデータを扱うための便利なツールを提供します。
> - 隣接行列を正規化してからGCNに入力することが重要です。PyTorch Geometricでは、この操作も簡単に行えます。

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.transforms import RandomLinkSplit

# サンプルデータの作成（ここではランダムデータを使用）
num_nodes = 1000
num_features = 10
num_classes = 2

# ノード特徴行列
x = torch.randn((num_nodes, num_features))
# ランダムな隣接行列（グラフの構造）
edge_index = torch.randint(0, num_nodes, (2, 5000))
# ラベル（0または1）
y = torch.randint(0, num_classes, (num_nodes,))

# PyTorch GeometricのDataオブジェクト
data = Data(x=x, edge_index=edge_index, y=y)

# トレーニングセットとテストセットに分割
transform = RandomLinkSplit(is_undirected=True, split_labels=True, num_val=0.0, num_test=0.2)
train_data, val_data, test_data = transform(data)

# トレーニング、検証、テストマスクの作成
train_data.train_mask = torch.ones(train_data.num_nodes, dtype=torch.bool)
test_data.test_mask = torch.ones(test_data.num_nodes, dtype=torch.bool)

# GCNモデルの定義
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# モデルのインスタンス化
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# トレーニングループ
for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    out = model(train_data)
    loss = F.nll_loss(out[train_data.train_mask], train_data.y[train_data.train_mask])
    loss.backward()
    optimizer.step()

    # エポックごとにトレーニングセットの精度を計算
    model.eval()
    _, pred = model(train_data).max(dim=1)
    correct = int(pred[train_data.train_mask].eq(train_data.y[train_data.train_mask]).sum().item())
    train_acc = correct / train_data.train_mask.sum().item()
    print(f'Epoch {epoch + 1}, Loss: {loss.item()}, Training Accuracy: {train_acc:.4f}')

# テストセットでの評価
model.eval()
_, pred = model(test_data).max(dim=1)
correct = int(pred[test_data.test_mask].eq(test_data.y[test_data.test_mask]).sum().item())
test_acc = correct / test_data.test_mask.sum().item()
print(f'Test Accuracy: {test_acc:.4f}')


## 論文の引用データセットでGCN


In [ ]:
from torch_geometric.datasets import Planetoid

# データセットを読み込む
dataset = Planetoid(root="./Cora", name="Cora")

# グラフ構造の数
print(len(dataset))

# クラス数
print(dataset.num_classes)

# 徴量の次元数(1433種類の特定のワードが論文中に含まれているか. 0:ない,1:ある)
print(dataset.num_node_features)

# ノード数
print(dataset[0].num_nodes)


In [ ]:
from torch_geometric.data import Data
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

# ノード特徴行列
x = dataset[0].x
# 隣接行列
edge_index = dataset[0].edge_index
# ラベル
y = dataset[0].y
# クラス数
num_features = dataset.num_node_features
# ラベル数
num_classes = dataset.num_classes

# データオブジェクト
data = Data(x=x, edge_index=edge_index, y=y)
transform = RandomLinkSplit(
    is_undirected=True, split_labels=True, num_val=0.0, num_test=0.2
)
train_data, _, test_data = transform(data)

# マスクの作成
train_data.train_mask = torch.ones(train_data.num_nodes, dtype=torch.bool)
test_data.test_mask = torch.ones(test_data.num_nodes, dtype=torch.bool)


class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


# モデルのインスタンス化
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# トレーニングループ
for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    out = model(train_data)
    loss = F.nll_loss(out[train_data.train_mask], train_data.y[train_data.train_mask])
    loss.backward()
    optimizer.step()

    # エポックごとにトレーニングセットの精度を計算
    model.eval()
    _, pred = model(train_data).max(dim=1)
    correct = int(
        pred[train_data.train_mask].eq(train_data.y[train_data.train_mask]).sum().item()
    )
    train_acc = correct / train_data.train_mask.sum().item()
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}, Training Accuracy: {train_acc:.4f}")

# テストセットでの評価
model.eval()
_, pred = model(test_data).max(dim=1)
correct = int(
    pred[test_data.test_mask].eq(test_data.y[test_data.test_mask]).sum().item()
)
test_acc = correct / test_data.test_mask.sum().item()
print(f"Test Accuracy: {test_acc:.4f}")
